# Extractor de características musciales
Sistema diseñado con la finalidad de extraer varias características musicales de archivos *MusciXML*

Las características que extrae son:
 * Tonalidad
 * Acordes
 * Prograsiones de acordes (secuencias)
 

## Tonalidad
La tonalidad de una pieza musical se refiere al sistema de organización de la altura de las notas en el que los elementos guardan entre sí un orden jerárquico de mayor a menor importancia.
En la música total, la música progresa alejándose y regresando a las notas fundamentales, que son las que rigen la importancia relativa de todos los sonidos de una composición musical.

La tonalidad de una pieza suele describirse por la nota fundamental y la cualidad de la escala que se desarrolla al rededor de dicha nota fundamental.

### Nota fundamental
Es una de las 12 notas del sistema temperado sobre la cuál se desarrolla la idea musical.

Suele ser la nota más frecuente en una pieza y comúnmente la pieza comienza y termina con ella y/o con alguna de las notas del acorde construido sobre esta misma.

### Cualidad de la Escala
Es nombre que recibe la serie de notas que presentan en una pieza y es determinada por los intervalos que existen entre las notas y la fundamental.

Para cuestiones de este estudio se utilizan únicamente los modos griegos y la escala menor armónica como cualidades de escala. Los modos griegos son 7 (uno por nota) e incluyen dentro de si mismos a la escala mayor (modo jónico) y la escala menor (modo eólico)

Los cualidades utilizadas se definen a continuación utilizando distancias entre notas, ya sea la distancia mínima entre dos notas: semitono (ST), o dos distancias mínimas: tono completo (T).


|Cualidad           |I-II|II-III|III-IV|IV-V|V-VI|VI-VII|VII-I|
|-------------------|----|------|------|----|----|------|-----|
|Mayor (Jónico)     |T   |T     |ST    |T   |T   |T     |ST   |
|Dorico             |T   |ST    |T     |T   |T   |ST    |T    |
|Frigio             |ST  |T     |T     |T   |ST  |T     |T    |
|Lidio              |T   |T     |T     |ST  |T   |T     |ST   |
|Mixolidio          |T   |T     |ST    |T   |T   |ST    |T    |
|Menor (Eólico)     |T   |ST    |T     |T   |ST  |T     |T    |
|Menor armonica     |T   |ST    |T     |T   |ST  |3m    |ST   |
|Locrio             |ST  |T     |T     |ST  |T   |T     |T    |


### Identificación de nota fundamental
Para la identificación de la nota fundamental de una pieza musical se identifica la nota más frecuente en la pieza, sin embargo, no es suficiente con contar las veces que se presenta la nota, sino que se debe dar un peso a cada aparición, por lo que se utiliza la duración de la nota. Además, hay notas clave que ayudan a definir la nota fundamental de la tonalidad, por lo que estas deben de recibir un peso extra.

Para cada nota se suma la duración de cada aparición en la pieza. Si la aparición de la nota es al inicio del compás (tiempo fuerte) se le da un peso extra y sí es al inicio o fin de la pieza se le da un pezo extra aún más significativo.

### Identificación de cualidad (escala) de la tonalidad
En cuanto a la identificación de la cualidad o escala de la tonalidad se utiliza un acercamiento de perfiles tal como lo hace Temperley en su [Investigación](pdf_resources/temperley2002.pdf). 

Se utiliza un perfil normalizado (valores de 0 a 1) para cada escala con la nota fundamental identificada y se calcula la distancia euclidiana entre el vector de frecuencias normalizado y los perfiles. La más cercana es la cualidad identificada.



In [ ]:
def dominant_key(measures: [Measure]) -> Key:
    # note frequency calculation
    note_count = []

    for i in range(12):
        note_count.append(0)

    for measure in measures:
        for note in measure.notes:
            try:
                if not note.rest:
                    # Extra weight for first and last note
                    if (measures.index(measure) == 0) or (measures.index(measure) == len(measures) - 1):
                        if note.first:
                            note_count[note.chroma] += int(note.duration) * 6 if note.octave < 4 else int(note.duration)
                        else:
                            note_count[note.chroma] += int(note.duration)
                    # Normal weight for normal beats
                    else:
                        # Minor extra weight for strong beat note (first note in measure)
                        note_count[note.chroma] += int(note.duration) + 1 if note.first else int(note.duration)

            except KeyError:
                pass

    frequencies = [count / sum(note_count) for count in note_count]

    # Generate normalized key profiles based on the piece's notes normalized frequency
    if len(frequencies) != 12:
        raise Exception
    offset = frequencies.index(max(frequencies))
    profiles = displace_profiles_by_key(offset, kb.NORMALIZED_PROFILES)

    # Calculate euclidean distances between the piece's notes frequency and the key profiles
    distances = profiles_distance(frequencies, profiles)

    key = Key(
        frequencies.index(max(frequencies)),
        list(distances.keys())[list(distances.values()).index(min(distances.values()))]
    )

    return key


In [ ]:
def displace_profiles_by_key(offset: int, profiles_dict: dict or list) -> dict or list:
    if isinstance(profiles_dict, list):
        profile = []
        [profile.append(prefix) for prefix in profiles_dict[-offset:]]
        [profile.append(suffix) for suffix in profiles_dict[0:-offset]]
        return profile
    else:
        profiles = {}
        for profile in profiles_dict:
            profiles[profile] = []
            [profiles[profile].append(prefix) for prefix in profiles_dict[profile][-offset:]]
            [profiles[profile].append(suffix) for suffix in profiles_dict[profile][0:-offset]]
        return profiles


def profiles_distance(main_profile: list[float], profiles_dict: dict) -> dict:
    distances = {}
    for profile in profiles_dict:
        squared_dist = 0
        for index in range(len(profiles_dict[profile])):
            squared_dist += (main_profile[index] - profiles_dict[profile][index]) ** 2
        distances[profile] = squared_dist ** (1 / 2)
    return distances


## Acordes
Un acorde está definido por el conjunto de tres notas o más separadas por intervalos.

Para identificar los acrodes se utiliza un acercamiento similar al de la tonalidad.

Los acordes se idetifican en un contexto de tiempo o a lo mucho de compás. 

Se analiza individualmente cada compás, tomando en cuenta la tonalidad previamente identificada.

Cada compás es dividido en tiempos según el signo de compás más reciente.

En cada tiempo se analizan las notas que pertenecen a dicho tiempo. Si las notas no forman un acorde se amplia el contexto sumando las notas del siguiente tiempo en caso de estar analizando el primer tiempo del compás, o las notas del tiempo anterior si se analiza cualquier otro tiempo que no sea el primero del compás.

Cada conjunto de notas es analizado para identificar primeramente la raíz del acorde y después la cualidad del acorde.

El algoritmo es capas de identificar 12 diferentes cualidades de acordes:
 * Mayor
 * menor
 * Aumentado
 * Disminuido
 * Séptima menor / Dominante con septima
 * Séptima mayor
 * Menor con séptima mayor
 * Menor con séptima
 * Aumentado con séptima mayor
 * Aumentado con séptima
 * Semidisminuido
 * Completamente disminuido
 
Se generan los posibles acordes utilizando las notas de la escala.

Se generan los perfiles de los acordes posibles usando las definiciones de los acordes y el perfil de la escala.

Se calcula la distancia euclidiana entre el perfil del conjunto de notas que se está analizando y los posibles acordes. Se selecciona sólo un grupo de tamaño predefinido los más cercanos.

Se cuentan las apariciones de cada raíz de los acordes cercanos y se normalizan.

Se multiplican las fecuencias normalizadas de acordes cercanos y las fecuencias de las notas en el conjunto de notas a analizar, aleiminando así posibles acordes que no presenten su nota fundamental en las notas analizadas.

Se toma el producto más grande como nota fundamental, sí existen más de un producto con el mismo valor, siendo este el máximo, se multiplica los productos por el perfil normalizado de la escala, eligiendo así la notá más probable de acuerdo al pefil de la escla de la pieza.



Una vez elegida la nota fundamental, se generan todos los acordes a partir de dicha nota, y se calcula la distancia ecluidiana a cada una desde el conjunto de notas que se analiza. Se elige al más cercano.


## Progresión de acordes

Lista de listas de acordes por orden de aparición en cada compás.

Se puede mostrar por nota fundamental o por grado de la tonalidad.

Para realizar progresiones agnosticas de tonalidad se utiliza la expresión por grados, de manera que se puedan descubrir secuecnias comunes entre las diferentes partituras sin considerar la tonalidad.
